## Gathering Labels for Training

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
from itertools import count
import natsort

In [2]:
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
import numpy
import random
import pandas as pd
import tqdm
from sklearn import metrics
import matplotlib.pyplot as plt

In [3]:
train_transforms = A.Compose(
    [
        A.Resize(224,224),
        A.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
        ToTensorV2(),
    ]
)

In [4]:
train_image_paths = []

In [5]:
for i in range(1,71):
    filename = '/home/zo2151/assignments/Data/Video%i'%(i,)
    train_image_paths.append(glob.glob(filename + '/*'))
train_image_paths1 = [item for sublist in train_image_paths for item in sublist]
train_image_paths1 = natsort.natsorted(train_image_paths1)

In [6]:
df = pd.read_csv("/home/zo2151/Processed_data.csv")
df1 = df.loc[:,"Phases"].to_numpy()
df2 = df1.tolist()
percentile_list = pd.DataFrame(
    {'Link': train_image_paths1,
     'Label': df2,
    })

In [7]:
percentile_list1 = percentile_list.sample(frac=1, random_state=1)
train_image_paths = percentile_list1.loc[:,"Link"].to_numpy().tolist()

In [8]:
labels = percentile_list1.loc[:,"Label"].to_numpy().tolist()
train_image_paths, valid_image_paths = train_image_paths[:int(0.8*len(train_image_paths))], train_image_paths[int(0.8*len(train_image_paths)):]  
train_labels, valid_labels = labels[:int(0.8*len(labels))], labels[int(0.8*len(labels)):] 

## Sampler

In [9]:
summary = {i:0 for i in range(14)}
num_classes = 14
total_samples = 0
for i in train_labels:
    total_samples += 1
    summary[i] += 1

class_weights = [total_samples/summary[i] for i in range(num_classes)]
weights = [class_weights[train_labels[i]] for i in range(total_samples)]
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), len(weights))

In [10]:
class SurgicalDataset(Dataset):
    def __init__(self, image_paths, labels, transform=False):
        super(SurgicalDataset, self).__init__()
        self.image_paths = image_paths
        self.transform = transform
        self.labels = labels
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        
        label = self.labels[idx]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label


In [11]:
train_dataset = SurgicalDataset(train_image_paths,train_labels, train_transforms)
val_dataset = SurgicalDataset(valid_image_paths,valid_labels, train_transforms)
train_loader = DataLoader(
    train_dataset, batch_size=1024, sampler= sampler
)

valid_loader = DataLoader(
    val_dataset, batch_size=1024, shuffle=True
)

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [13]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 14)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=14, bias=True)
)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [15]:
for epoch in range(5):  # loop over the dataset multiple times
    t = time.time()
    running_loss = 0.0
    loop = tqdm.tqdm(train_loader, total = len(train_loader), leave = True)
    for img, label in loop:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = img.to(device), label.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    d = time.time()-t
    print(d)
    torch.save({
            'epoch': 2,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, "/home/zo2151/model2.pt")
print('Finished Training')

100%|█████████████████████████████████████████████████████████████████████████| 169/169 [18:47<00:00,  6.67s/it]


1127.7954268455505


100%|█████████████████████████████████████████████████████████████████████████| 169/169 [18:47<00:00,  6.67s/it]


1127.2274136543274


100%|█████████████████████████████████████████████████████████████████████████| 169/169 [18:48<00:00,  6.68s/it]


1128.2046687602997


100%|█████████████████████████████████████████████████████████████████████████| 169/169 [18:47<00:00,  6.67s/it]


1127.279236316681


100%|█████████████████████████████████████████████████████████████████████████| 169/169 [18:47<00:00,  6.67s/it]


1127.4367802143097
Finished Training


In [16]:
classes = [i for i in range(14)]
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
pr = []
pred = []
l = []
# again no gradients needed
t = time.time()
with torch.no_grad():
    for data in valid_loader:
        images, labels = data[0].to(device), data[1].to(device)
        l.append(labels)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        m = F.softmax(outputs, dim=1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            pred.append(prediction)
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
        for p in m:
            pr.append(p)
print(time.time()-t)
print(correct_pred)
print(total_pred)
# print accuracy for each class
#for classname, correct_count in correct_pred.items():
    #accuracy = 100 * float(correct_count) / total_pred[classname]
    #print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

283.9022831916809
{0: 42, 1: 17, 2: 403, 3: 4089, 4: 69, 5: 279, 6: 7361, 7: 1930, 8: 2691, 9: 4630, 10: 237, 11: 6601, 12: 392, 13: 1909}
{0: 52, 1: 18, 2: 587, 3: 5470, 4: 213, 5: 288, 6: 11135, 7: 2177, 8: 3001, 9: 5799, 10: 247, 11: 10405, 12: 404, 13: 3216}


In [17]:
for i in range(len(l)):
    l[i] = l[i].cpu()
for i in range(len(l)):
    l[i] = l[i].data.numpy()
l = [item for sublist in l for item in sublist]
for i in range(len(l)):
    pred[i] = pred[i].cpu().data.numpy()
for i in range(len(l)):
    pr[i] = pr[i].cpu().data.numpy()

## Some metrics

In [18]:
metrics.accuracy_score(l, pred)

0.7125918348367898

In [19]:
metrics.f1_score(l, pred, average="macro")

0.7677966238682671

In [20]:
metrics.precision_score(l, pred, average=None)

array([0.97674419, 0.80952381, 0.94158879, 0.95716292, 0.84146341,
       0.85846154, 0.92626148, 0.82726104, 0.29038524, 0.79525936,
       0.94047619, 0.9561124 , 0.90114943, 0.39110838])

In [21]:
metrics.recall_score(l, pred, average=None)

array([0.80769231, 0.94444444, 0.68654174, 0.74753199, 0.32394366,
       0.96875   , 0.6610687 , 0.88654111, 0.8967011 , 0.79841352,
       0.95951417, 0.63440654, 0.97029703, 0.59359453])

In [22]:
auc = metrics.roc_auc_score(l, pr, multi_class = "ovr")

In [23]:
auc

0.9787734785339363

In [24]:
auc = metrics.roc_auc_score(l, pr, multi_class = "ovo")

In [25]:
auc

0.9831764253488794

In [26]:
mem_params = sum([param.nelement()*param.element_size() for param in net.parameters()])
mem_bufs = sum([buf.nelement()*buf.element_size() for buf in net.buffers()])
mem = mem_params + mem_bufs

In [27]:
mem

21630504